In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key = os.getenv('GEMINI_KEY'))



In [7]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
from typing import Annotated, TypedDict
import operator
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


In [8]:
def multiply(a:int, b:int)->int:
    """
    Multiply a and b
    Args:
        a:first int 
        b: second int   
    """
    return a*b

In [13]:
def addition(a:int, b:int)->int:
    """
    Add a and b
    Args:
        a:first int 
        b: second int   
    """
    return a+b

In [9]:
def divide(a:int, b:int)->int:
    """
    Divide a and b
    Args:
        a:first int 
        b: second int   
    """
    return a/b

In [11]:
search = DuckDuckGoSearchRun()

In [38]:
import yfinance as yf
def get_stock_price(ticker:str)->float:
    """Get the stock price from yahoo finnance
    
    Args
    ticker:str
    """
    stock = yf.Ticker(ticker)
    return stock.info['previousClose']

In [45]:
get_stock_price("AAPL")

188.38

In [39]:
tools = [multiply, addition, divide, search, get_stock_price]

In [40]:
lllm_with_tools = llm.bind_tools(tools)

In [16]:
system_prompt = SystemMessage(content="You are a helpful assistant tasked with using search and performing arithmetic on a set of inputs.")

In [51]:
def reasoner(state):
    query = state['query']
    messages = state['messages']
    message = HumanMessage(content=query)
    messages.append(message)
    return {"messages":[lllm_with_tools.invoke([system_prompt]+messages)]}

In [52]:
class Graphstate(TypedDict):
    """State of the Graph"""
    query:str
    finance:str
    final_ans:str
    messages:Annotated[list[AnyMessage], operator.add]



In [53]:
workflow = StateGraph(Graphstate)
workflow.add_node("reasoner", reasoner)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, 'reasoner')
workflow.add_conditional_edges(
    "reasoner",

    tools_condition
)
workflow.add_edge('tools', 'reasoner')

app= workflow.compile()

In [81]:
result = app.invoke({'query':"give ipl todays match details  ", 'nessages':[]})

In [82]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

give ipl todays match details  
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_search (bbbc52b3-bff3-4a39-bb47-828419531c7c)
 Call ID: bbbc52b3-bff3-4a39-bb47-828419531c7c
  Args:
    query: ipl todays match details
================================= Tool Message =================================
Name: duckduckgo_search

Live cricket scores and updates for IPL 2025 and all current top-level cricket matches for men and women - Tests, ODIs, T20s, first-class cricket, List A cricket, and more. Get real-time IPL live scores & cricket match updates on IPL.com! Stay updated with ball-by-ball commentary, instant scorecards, and in-depth analysis for all major cricket tournaments. ... IPL Trending News. ... Entertainers Cricket League 2025 All Teams, Squads, Captains, and Auction Highlights. IPL 2025 Tickets Booking: Online Details ... The IPL 2025 match o